In [13]:
import os
import numpy as np
import pandas as pd
from scipy import stats
from scipy.signal import find_peaks
from scipy.fft import fft
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

# Function to create directory if it doesn't exist
def create_dir_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

# Time-domain features
def extract_time_domain_features(data):
    features = {}
    
    # Extract features for each axis
    for axis in ['x_norm', 'y_norm', 'z_norm']:
        axis_data = data[axis].values
        
        # Basic statistics
        features[f'{axis}_mean'] = np.mean(axis_data)
        features[f'{axis}_median'] = np.median(axis_data)
        features[f'{axis}_std'] = np.std(axis_data)
        features[f'{axis}_var'] = np.var(axis_data)
        features[f'{axis}_min'] = np.min(axis_data)
        features[f'{axis}_max'] = np.max(axis_data)
        features[f'{axis}_range'] = np.max(axis_data) - np.min(axis_data)
        features[f'{axis}_rms'] = np.sqrt(np.mean(np.square(axis_data)))
        
        # Distribution shape
        features[f'{axis}_skewness'] = stats.skew(axis_data)
        features[f'{axis}_kurtosis'] = stats.kurtosis(axis_data)
        features[f'{axis}_peak_to_peak'] = np.max(axis_data) - np.min(axis_data)
        
        # Energy
        features[f'{axis}_energy'] = np.sum(np.square(axis_data)) / len(axis_data)
        
        # Zero crossing rate
        zero_crossings = np.where(np.diff(np.signbit(axis_data)))[0]
        features[f'{axis}_zero_crossing_rate'] = len(zero_crossings) / len(axis_data)
    
    # Signal Magnitude Area (SMA)
    sma = np.mean(np.abs(data['x_norm']) + np.abs(data['y_norm']) + np.abs(data['z_norm']))
    features['sma'] = sma
    
    return features

# Frequency-domain features
def extract_frequency_domain_features(data, sampling_rate=100):  # Assuming 100Hz, adjust if different
    features = {}
    
    for axis in ['x_norm', 'y_norm', 'z_norm']:
        axis_data = data[axis].values
        
        # Compute FFT
        n = len(axis_data)
        fft_values = fft(axis_data)
        fft_magnitude = np.abs(fft_values[:n//2])
        freqs = np.fft.fftfreq(n, 1/sampling_rate)[:n//2]
        
        # Dominant frequency
        if len(fft_magnitude) > 0:
            features[f'{axis}_dominant_freq'] = freqs[np.argmax(fft_magnitude)]
        else:
            features[f'{axis}_dominant_freq'] = 0
            
        # Spectral energy
        features[f'{axis}_spectral_energy'] = np.sum(np.square(fft_magnitude)) / n
        
        # Spectral entropy
        if np.sum(fft_magnitude) > 0:
            prob = fft_magnitude / np.sum(fft_magnitude)
            # Add small value to avoid log(0)
            spectral_entropy = -np.sum(prob * np.log2(prob + 1e-10))
            features[f'{axis}_spectral_entropy'] = spectral_entropy
        else:
            features[f'{axis}_spectral_entropy'] = 0
        
        # Spectral centroid
        if np.sum(fft_magnitude) > 0:
            features[f'{axis}_spectral_centroid'] = np.sum(freqs * fft_magnitude) / np.sum(fft_magnitude)
        else:
            features[f'{axis}_spectral_centroid'] = 0
        
        # Band power (example: 0-10 Hz)
        low_freq = 0
        high_freq = 10
        band_indices = np.where((freqs >= low_freq) & (freqs <= high_freq))[0]
        if len(band_indices) > 0:
            features[f'{axis}_band_power_0_10Hz'] = np.sum(np.square(fft_magnitude[band_indices])) / len(band_indices)
        else:
            features[f'{axis}_band_power_0_10Hz'] = 0
        
        # First k FFT coefficients (e.g., first 5)
        k = min(5, len(fft_magnitude))
        for i in range(k):
            if i < len(fft_magnitude):
                features[f'{axis}_fft_coef_{i}'] = fft_magnitude[i]
            else:
                features[f'{axis}_fft_coef_{i}'] = 0
                
    return features

# Magnitude-based features
def extract_magnitude_features(data):
    features = {}
    
    # Calculate magnitude
    magnitude = np.sqrt(np.square(data['x_norm']) + np.square(data['y_norm']) + np.square(data['z_norm']))
    
    # Statistics on magnitude
    features['magnitude_mean'] = np.mean(magnitude)
    features['magnitude_var'] = np.var(magnitude)
    features['magnitude_rms'] = np.sqrt(np.mean(np.square(magnitude)))
    features['magnitude_max'] = np.max(magnitude)
    features['magnitude_min'] = np.min(magnitude)
    features['magnitude_range'] = np.max(magnitude) - np.min(magnitude)
    
    return features

# Cross-axis features
def extract_cross_axis_features(data):
    features = {}
    
    # Correlation coefficients
    corr_xy = np.corrcoef(data['x_norm'], data['y_norm'])[0, 1]
    corr_xz = np.corrcoef(data['x_norm'], data['z_norm'])[0, 1]
    corr_yz = np.corrcoef(data['y_norm'], data['z_norm'])[0, 1]
    
    features['correlation_xy'] = corr_xy
    features['correlation_xz'] = corr_xz
    features['correlation_yz'] = corr_yz
    
    # Covariance between axes
    features['covariance_xy'] = np.cov(data['x_norm'], data['y_norm'])[0, 1]
    features['covariance_xz'] = np.cov(data['x_norm'], data['z_norm'])[0, 1]
    features['covariance_yz'] = np.cov(data['y_norm'], data['z_norm'])[0, 1]
    
    # Angles between acceleration vectors (relative angles)
    # Using dot product formula: cos(θ) = (a·b) / (|a|·|b|)
    # Taking mean vectors for simplicity
    x_mean = np.mean(data['x_norm'])
    y_mean = np.mean(data['y_norm'])
    z_mean = np.mean(data['z_norm'])
    
    # Create vectors
    vec1 = np.array([x_mean, y_mean, 0])
    vec2 = np.array([x_mean, 0, z_mean])
    vec3 = np.array([0, y_mean, z_mean])
    
    # Compute norms
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    norm3 = np.linalg.norm(vec3)
    
    # Calculate angles
    if norm1 > 0 and norm2 > 0:
        dot_product = np.dot(vec1, vec2)
        cos_angle = dot_product / (norm1 * norm2)
        # Clip to handle floating point errors
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        features['angle_xy_xz'] = np.arccos(cos_angle) * 180 / np.pi
    else:
        features['angle_xy_xz'] = 0
        
    if norm1 > 0 and norm3 > 0:
        dot_product = np.dot(vec1, vec3)
        cos_angle = dot_product / (norm1 * norm3)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        features['angle_xy_yz'] = np.arccos(cos_angle) * 180 / np.pi
    else:
        features['angle_xy_yz'] = 0
        
    if norm2 > 0 and norm3 > 0:
        dot_product = np.dot(vec2, vec3)
        cos_angle = dot_product / (norm2 * norm3)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        features['angle_xz_yz'] = np.arccos(cos_angle) * 180 / np.pi
    else:
        features['angle_xz_yz'] = 0
    
    return features

# Windowed/Segmented features
def extract_windowed_features(data, window_size=100):
    features = {}
    
    # If data is smaller than window size, use whole data
    if len(data) < window_size:
        window_size = len(data)
        
    # Create windows
    num_windows = len(data) // window_size
    
    # Initialize counters
    total_peaks = 0
    total_troughs = 0
    window_slopes = []
    
    for i in range(num_windows):
        start_idx = i * window_size
        end_idx = (i + 1) * window_size
        
        window_data = data.iloc[start_idx:end_idx]
        
        # Calculate slope for each axis in this window
        for axis in ['x_norm', 'y_norm', 'z_norm']:
            axis_data = window_data[axis].values
            
            if len(axis_data) > 1:
                # Linear regression for slope
                x = np.arange(len(axis_data))
                slope, _, _, _, _ = stats.linregress(x, axis_data)
                window_slopes.append(slope)
            
            # Find peaks and troughs
            peaks, _ = find_peaks(axis_data)
            troughs, _ = find_peaks(-axis_data)
            
            total_peaks += len(peaks)
            total_troughs += len(troughs)
    
    # Average features across windows
    features['avg_window_slope'] = np.mean(window_slopes) if window_slopes else 0
    features['total_peaks'] = total_peaks
    features['total_troughs'] = total_troughs
    features['peaks_per_window'] = total_peaks / max(1, num_windows)
    features['troughs_per_window'] = total_troughs / max(1, num_windows)
    
    # Activity counts (accumulated counts over a threshold)
    threshold = 0.5  # Adjust as needed
    activity_counts = np.sum(np.abs(data['x_norm']) > threshold) + \
                      np.sum(np.abs(data['y_norm']) > threshold) + \
                      np.sum(np.abs(data['z_norm']) > threshold)
    features['activity_counts'] = activity_counts
    
    return features

# Advanced features
def extract_advanced_features(data):
    features = {}
    
    # Signal Vector Magnitude (SVM)
    svm = np.mean(np.sqrt(np.square(data['x_norm']) + 
                          np.square(data['y_norm']) + 
                          np.square(data['z_norm'])))
    features['svm'] = svm
    
    # Tilt angles (assuming z is vertical)
    # Convert to radians and then degrees
    x_mean = np.mean(data['x_norm'])
    y_mean = np.mean(data['y_norm'])
    z_mean = np.mean(data['z_norm'])
    
    # Calculate tilt angles
    if z_mean != 0:
        features['tilt_x'] = np.arctan(x_mean / z_mean) * 180 / np.pi
    else:
        features['tilt_x'] = 90 if x_mean > 0 else -90
        
    if z_mean != 0:
        features['tilt_y'] = np.arctan(y_mean / z_mean) * 180 / np.pi
    else:
        features['tilt_y'] = 90 if y_mean > 0 else -90
    
    # Jerk (rate of change of acceleration) - first derivative
    jerk_x = np.diff(data['x_norm'])
    jerk_y = np.diff(data['y_norm'])
    jerk_z = np.diff(data['z_norm'])
    
    if len(jerk_x) > 0:
        features['jerk_x_mean'] = np.mean(np.abs(jerk_x))
        features['jerk_y_mean'] = np.mean(np.abs(jerk_y))
        features['jerk_z_mean'] = np.mean(np.abs(jerk_z))
        features['jerk_magnitude'] = np.mean(np.sqrt(np.square(jerk_x) + 
                                                    np.square(jerk_y) + 
                                                    np.square(jerk_z)))
    else:
        features['jerk_x_mean'] = 0
        features['jerk_y_mean'] = 0
        features['jerk_z_mean'] = 0
        features['jerk_magnitude'] = 0
    
    # Area under curve (AUC) - trapezoidal integration
    features['auc_x'] = np.trapz(np.abs(data['x_norm']))
    features['auc_y'] = np.trapz(np.abs(data['y_norm']))
    features['auc_z'] = np.trapz(np.abs(data['z_norm']))
    
    # Sample entropy (measure of complexity) - approximation using std
    features['entropy_x'] = np.log(2 * np.pi * np.e * np.std(data['x_norm'])**2) / 2 if np.std(data['x_norm']) > 0 else 0
    features['entropy_y'] = np.log(2 * np.pi * np.e * np.std(data['y_norm'])**2) / 2 if np.std(data['y_norm']) > 0 else 0
    features['entropy_z'] = np.log(2 * np.pi * np.e * np.std(data['z_norm'])**2) / 2 if np.std(data['z_norm']) > 0 else 0
    
    # Auto-correlation at lag k (for periodicity detection)
    lag = 1  # Adjust as needed
    if len(data) > lag:
        features['autocorr_x'] = np.corrcoef(data['x_norm'][:-lag], data['x_norm'][lag:])[0, 1]
        features['autocorr_y'] = np.corrcoef(data['y_norm'][:-lag], data['y_norm'][lag:])[0, 1]
        features['autocorr_z'] = np.corrcoef(data['z_norm'][:-lag], data['z_norm'][lag:])[0, 1]
    else:
        features['autocorr_x'] = 0
        features['autocorr_y'] = 0
        features['autocorr_z'] = 0
    
    return features

# Function to extract all features
def extract_all_features(data_file):
    try:
        # Read data
        data = pd.read_csv(data_file)
        
        # Check if file has the required columns
        required_columns = ['x_norm', 'y_norm', 'z_norm']
        if not all(col in data.columns for col in required_columns):
            print(f"Skipping {data_file} - missing required columns")
            return None, None
            
        # Get filename without directory and extension
        file_basename = os.path.basename(data_file)
        
        # Extract all features
        features = {}
        
        # Add file information
        features['file_name'] = file_basename
        features['char_name'] = os.path.splitext(file_basename)[0]
        
        # Time domain features
        time_features = extract_time_domain_features(data)
        features.update(time_features)
        
        # Frequency domain features
        freq_features = extract_frequency_domain_features(data)
        features.update(freq_features)
        
        # Magnitude features
        mag_features = extract_magnitude_features(data)
        features.update(mag_features)
        
        # Cross-axis features
        cross_features = extract_cross_axis_features(data)
        features.update(cross_features)
        
        # Windowed features
        window_features = extract_windowed_features(data)
        features.update(window_features)
        
        # Advanced features
        adv_features = extract_advanced_features(data)
        features.update(adv_features)
        
        return features, file_basename
        
    except Exception as e:
        print(f"Error processing {data_file}: {e}")
        return None, None

# Main function to traverse directory and extract features
def process_all_files(input_dir, output_dir):
    print(f"Starting to process files from: {input_dir}")
    
    # Create output directory if it doesn't exist
    create_dir_if_not_exists(output_dir)
    
    # List to store all features
    all_features = []
    file_count = 0
    
    # Traverse through all files in input directory and subdirectories
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.csv'):
                # Full path to file
                file_path = os.path.join(root, file)
                print(f"Processing: {file_path}")
                
                # Extract features
                features, file_basename = extract_all_features(file_path)
                
                if features is not None:
                    all_features.append(features)
                    file_count += 1
                    
                    # Also save individual feature file
                    char_name = os.path.splitext(file_basename)[0]
                    individual_output_path = os.path.join(output_dir, f"{char_name}_features.csv")
                    
                    # Save as individual file
                    pd.DataFrame([features]).to_csv(individual_output_path, index=False)
                    print(f"Saved individual features to: {individual_output_path}")
    
    print(f"Processed {file_count} files")
    
    # Save all features to a single file if any were extracted
    if all_features:
        all_features_df = pd.DataFrame(all_features)
        all_features_path = os.path.join(output_dir, "all_features.csv")
        all_features_df.to_csv(all_features_path, index=False)
        print(f"Saved combined features to: {all_features_path}")
    else:
        print("No features were extracted.")

# Main execution
if __name__ == "__main__":
    # Define input and output directories
    input_directory = "preprocessed_data"
    output_directory = "featureExtraction"
    
    # Process all files
    process_all_files(input_directory, output_directory)
    print("Feature extraction complete!")

Starting to process files from: preprocessed_data
Processing: preprocessed_data\অ\অ_36606b04_Right Hand_20250423_120617_preprocessed.csv
Saved individual features to: featureExtraction\অ_36606b04_Right Hand_20250423_120617_preprocessed_features.csv
Processing: preprocessed_data\অ\অ_36606b04_Right Hand_20250423_120731_preprocessed.csv
Saved individual features to: featureExtraction\অ_36606b04_Right Hand_20250423_120731_preprocessed_features.csv
Processing: preprocessed_data\আ\আ_36606b04_Right Hand_20250423_120748_preprocessed.csv
Saved individual features to: featureExtraction\আ_36606b04_Right Hand_20250423_120748_preprocessed_features.csv
Processing: preprocessed_data\আ\আ_36606b04_Right Hand_20250423_120805_preprocessed.csv
Saved individual features to: featureExtraction\আ_36606b04_Right Hand_20250423_120805_preprocessed_features.csv
Processing: preprocessed_data\ক\ক_01083526_Right Hand_20250423_122648_preprocessed.csv
Saved individual features to: featureExtraction\ক_01083526_Right Ha